## The Smart Recruits
### 23/07/2016
Rui Mendes


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime

In [5]:
def loadTrainAndTestDatasets():
    train_df = pd.read_csv("data/Train_pjb2QcD.csv")
    test_df = pd.read_csv("data/Test_wyCirpO.csv")
    return train_df, test_df


In [6]:
from datetime import date

def calculate_age(born):
    today = date.today()
    date_object = datetime.strptime(born, "%m/%d/%Y")
    born_year = date_object.year
    born_month = date_object.month
    born_day = date_object.day
    
    return today.year - born_year - ((today.month, today.day) < (born_month, born_day))


In [21]:
def prepareDatasets(train_df, test_df):
    
    print('-- Transformation step has begun --- ')
    ## JOIN train and test datasets
    all_data = pd.concat([train_df, test_df])
    
    train_length = train_df.shape[0]
    
    
    #Separating target variable
    train_outcome = train_df["Business_Sourced"]
    all_data.drop(["Business_Sourced"], axis=1, inplace=True)
    
    #Get Test IDs
    test_IDS = test_df['ID']

    # Drop columns
    all_data.drop(["ID","Manager_DOJ", "Manager_DoB"], axis=1, inplace=True)
    
    
    # Fill NA values
    all_data.loc[(all_data['Applicant_City_PIN'].isnull()),'Applicant_City_PIN'] = 'None'
    all_data.loc[(all_data['Applicant_BirthDate'].isnull()),'Applicant_BirthDate'] = all_data['Applicant_BirthDate'].mode()[0]
    all_data.loc[(all_data['Manager_Gender'].isnull()),'Manager_Gender'] = all_data['Manager_Gender'].mode()[0]
    all_data.loc[(all_data['Applicant_Gender'].isnull()),'Applicant_Gender'] = all_data['Applicant_Gender'].mode()[0]
    all_data.loc[(all_data['Applicant_Marital_Status'].isnull()),'Applicant_Marital_Status'] = 'None'
    all_data.loc[(all_data['Manager_Status'].isnull()),'Manager_Status'] = 'None'
    all_data.loc[(all_data['Applicant_Occupation'].isnull()),'Applicant_Occupation'] = 'Others'
    all_data.loc[(all_data['Manager_Joining_Designation'].isnull()),'Manager_Joining_Designation'] = 'Other'
    all_data.loc[(all_data['Manager_Current_Designation'].isnull()),'Manager_Current_Designation'] = 'Other'
    all_data.loc[(all_data['Applicant_Qualification'].isnull()),'Applicant_Qualification'] = 'Others'
    all_data.loc[(all_data['Manager_Business'].isnull()),'Manager_Business'] = all_data['Manager_Business'].mean()
    all_data.loc[(all_data['Manager_Business2'].isnull()),'Manager_Business2'] = all_data['Manager_Business2'].mean()
    all_data.loc[(all_data['Manager_Grade'].isnull()),'Manager_Grade'] = all_data['Manager_Grade'].mean()
    all_data.loc[(all_data['Manager_Num_Application'].isnull()),'Manager_Num_Application'] = all_data['Manager_Num_Application'].mean()
    all_data.loc[(all_data['Manager_Num_Coded'].isnull()),'Manager_Num_Coded'] = all_data['Manager_Num_Coded'].mean()
    all_data.loc[(all_data['Manager_Num_Products'].isnull()),'Manager_Num_Products'] = all_data['Manager_Num_Products'].mean()
    all_data.loc[(all_data['Manager_Num_Products2'].isnull()),'Manager_Num_Products2'] = all_data['Manager_Num_Products2'].mean()
    print('  * FILL NA VALUES is completed! ')
    
    
    # Application_Receipt_Date
    all_data['Application_Receipt_Date_Year'] = pd.DatetimeIndex(all_data['Application_Receipt_Date']).year
    all_data['Application_Receipt_Date_Month'] = pd.DatetimeIndex(all_data['Application_Receipt_Date']).month
    all_data['Application_Receipt_Date_Day'] = pd.DatetimeIndex(all_data['Application_Receipt_Date']).day
    all_data.drop(["Application_Receipt_Date"], axis=1, inplace=True)
    print('  * Application_Receipt_Date is completed! ')
    
    # Applicant_BirthDate    
    #all_data['Applicant_BirthDate_Year'] = pd.DatetimeIndex(all_data['Applicant_BirthDate']).year
    #all_data['Applicant_BirthDate_Month'] = pd.DatetimeIndex(all_data['Applicant_BirthDate']).month
    #all_data['Applicant_BirthDate_Day'] = pd.DatetimeIndex(all_data['Applicant_BirthDate']).day
    all_data["Applicant_Age"] = all_data['Applicant_BirthDate'].apply(calculate_age)
    all_data.drop(["Applicant_BirthDate"], axis=1, inplace=True)
    print('  * Applicant_BirthDate is completed! ')
    
    # Manager_Gender
    all_data["Manager_Gender"] = all_data['Manager_Gender'].apply(lambda e : 1 if "M" else 0 )
    print('  * Manager_Gender is completed! ')
    
    
    # Applicant_Gender
    all_data["Applicant_Gender"] = all_data['Applicant_Gender'].apply(lambda e : 1 if "M" else 0 )
    print('  * Applicant_Gender is completed! ')
    
    ########################################################################################################
    #all_data['is_holiday'] = all_data.DateTime.apply(is_holiday)
    #print('       # IS HOLIDAY is completed! ')
    #all_data['is_weekend'] = all_data.DateTime.apply(is_weekend)
    #print('       # IS WEEKEND is completed! ')
    #all_data['season'] = all_data.DateTime.apply(get_season)
    #print('       # SEASON is completed! ')
    
    #all_data["is_mix"] = all_data['Breed'].apply(lambda e : 1 if "Mix" in e else 0 )
    #print('  * BREED is completed! ')
    ########################################################################################################
    
    #Normalize columns
    cols_to_norm = ['Manager_Grade', 'Manager_Business', 'Manager_Business2', 'Manager_Grade', 
                    'Manager_Num_Application', 'Manager_Num_Coded', 'Manager_Num_Products', 
                    'Manager_Num_Products2', 'Application_Receipt_Date_Year', 
                    'Application_Receipt_Date_Month', 'Application_Receipt_Date_Day',
                    'Applicant_Age', 
                   #'Applicant_BirthDate_Year', 'Applicant_BirthDate_Month', 'Applicant_BirthDate_Day'
                   ]
    all_data[cols_to_norm] = all_data[cols_to_norm].apply(lambda x: (x - x.mean()) / (x.max() - x.min()))
    
    
    # http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html
    #Encode the categorical data of all_data, column by column due memory restrictions
    Office_PIN_Status_encoded = pd.get_dummies(all_data['Office_PIN'], columns='Office_PIN', prefix='Office_PIN')
    Applicant_Marital_Status_encoded = pd.get_dummies(all_data['Applicant_Marital_Status'], columns='Applicant_Marital_Status', 
                                                      prefix='Applicant_Marital_Status')
    Applicant_Occupation_encoded = pd.get_dummies(all_data['Applicant_Occupation'], columns='Applicant_Occupation'
                                                 , prefix='Applicant_Occupation')
    Manager_Status_encoded = pd.get_dummies(all_data['Manager_Status'], columns='Manager_Status', prefix='Manager_Status')
    Applicant_City_PIN_encoded = pd.get_dummies(all_data['Applicant_City_PIN'], columns='Applicant_City_PIN', 
                                                prefix='Applicant_City_PIN')
    
    Applicant_Qualification_encoded = pd.get_dummies(all_data['Applicant_Qualification'], columns='Applicant_Qualification'
                                                 , prefix='Applicant_Qualification')
    Manager_Joining_Designation_encoded = pd.get_dummies(all_data['Manager_Joining_Designation'], columns='Manager_Joining_Designation'
                                                 , prefix='Manager_Joining_Designation')
    Manager_Current_Designation_encoded = pd.get_dummies(all_data['Manager_Current_Designation'], columns='Manager_Current_Designation'
                                                 , prefix='Manager_Current_Designation')
    
    print('  * GET DUMMIES is completed! ')
    
    # http://pandas.pydata.org/pandas-docs/stable/merging.html
    all_data.drop(["Office_PIN"], axis=1, inplace=True)
    all_data.drop(["Applicant_Marital_Status"], axis=1, inplace=True)
    all_data.drop(["Applicant_Occupation"], axis=1, inplace=True)
    all_data.drop(["Manager_Status"], axis=1, inplace=True)
    all_data.drop(["Applicant_City_PIN"], axis=1, inplace=True)
    all_data.drop(["Applicant_Qualification"], axis=1, inplace=True)
    all_data.drop(["Manager_Joining_Designation"], axis=1, inplace=True)
    all_data.drop(["Manager_Current_Designation"], axis=1, inplace=True)
    print('  * DROP GET DUMMIES is completed! ')
    
    all_data_encoded = pd.concat([all_data, Office_PIN_Status_encoded, Applicant_Marital_Status_encoded, 
                                  Applicant_Occupation_encoded, Manager_Status_encoded, 
                                  Applicant_City_PIN_encoded, Applicant_Qualification_encoded,
                                  Manager_Joining_Designation_encoded, Manager_Current_Designation_encoded], axis=1)
    
    
    #Split again for train and test
    train = all_data_encoded[:train_length]
    test = all_data_encoded[train_length:]
    
    print('-- Transformation step has finished --- ')
    
    return train_outcome, test_IDS, train, test

## Cross-validation metrics

In [22]:
from sklearn import metrics
from sklearn.cross_validation import train_test_split

def calculateCVMetrics(train, train_outcome, model):
    
    #Training a RF to get some metrics
    X_train, X_val, y_train, y_val = train_test_split(train, train_outcome, test_size=0.3)
    y_pred_val = makePredictions(model, X_val)

    print('ROC AUC score')
    print(metrics.roc_auc_score(y_val, y_pred_val))


## Create Model

In [46]:
from sklearn.ensemble import RandomForestClassifier

def CreateFinalModel(train, train_outcome):
    
    model = RandomForestClassifier(n_estimators=5000,n_jobs=5)
    model.fit(train, train_outcome)
    
    return model

## Create predictions

In [24]:
def makePredictions(model, test):
    
    return model.predict(test) 

## Create Submission File

In [25]:
from datetime import datetime

def createCSVSubmissionFile(predictions, ids):
    
    results = pd.read_csv("submissions/Sample_Submission_RHYDQa4.csv")
    results['ID'], results['Business_Sourced'] = ids.values, predictions
    
    now = datetime.now().strftime("%Y%m%d-%H%M%S")
    fileName = "submission_" + now + ".csv"
    results.to_csv("submissions/" + fileName, index=False)


# RUN ALL STEPS

In [26]:
train_df, test_df = loadTrainAndTestDatasets()

In [27]:
train_outcome, test_IDS, train, test = prepareDatasets(train_df, test_df)

-- Transformation step has begun --- 
  * FILL NA VALUES is completed! 
  * Application_Receipt_Date is completed! 
  * Applicant_BirthDate is completed! 
  * Manager_Gender is completed! 
  * Applicant_Gender is completed! 
  * GET DUMMIES is completed! 
  * DROP GET DUMMIES is completed! 
-- Transformation step has finished --- 


In [28]:
len(train.columns)

3908

In [29]:
#train.head()

In [30]:
#test.head()

In [31]:
finalModel = CreateFinalModel(train, train_outcome)

In [32]:
calculateCVMetrics(train, train_outcome, finalModel)

ROC AUC score
0.999495459132


In [33]:
predictions = makePredictions(finalModel, test)

In [34]:
createCSVSubmissionFile(predictions, test_IDS)

In [35]:
print(sorted(zip(finalModel.feature_importances_, list(train.columns)), reverse=True)[:10])

[(0.070699541886896131, 'Applicant_Age'), (0.061784388141541427, 'Application_Receipt_Date_Day'), (0.04242086090859095, 'Application_Receipt_Date_Month'), (0.035366896415048976, 'Manager_Business'), (0.035356065453950709, 'Manager_Business2'), (0.032106781221587985, 'Manager_Num_Application'), (0.026987026905159129, 'Manager_Num_Products'), (0.026949925593850187, 'Manager_Num_Products2'), (0.020274434791873019, 'Manager_Num_Coded'), (0.013674191102223504, 'Manager_Grade')]


# TESTING FEATURE SELECTION

In [47]:
from sklearn.feature_selection import SelectFromModel
model = SelectFromModel(finalModel, prefit=True)
X_new = model.transform(train)

In [48]:
newModel = CreateFinalModel(X_new, train_outcome)

In [49]:
calculateCVMetrics(X_new, train_outcome, newModel)

ROC AUC score
0.998944033791


In [50]:
Y_new = model.transform(test)
new_predictions = makePredictions(newModel, Y_new)

In [51]:
createCSVSubmissionFile(new_predictions, test_IDS)